In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotnine as pn 
from os.path import exists

Download FBI crime data

In [14]:
# each xls sheet has that year and the previous year. Download 2010, 2012, 2014, 2016. 2018, 2019
urls = ['https://ucr.fbi.gov/crime-in-the-u.s/2010/crime-in-the-u.s.-2010/tables/10tbl04.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2012/crime-in-the-u.s.-2012/tables/4tabledatadecoverviewpdf/table_4_crime_in_the_united_states_by_region_geographic_division_and_state_2011-2012.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2014/crime-in-the-u.s.-2014/tables/table-4/table_4_crime_in_the_united_states_by_region_geographic_division_and_state_2013-2014.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2016/crime-in-the-u.s.-2016/tables/table-2/table-2.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2018/crime-in-the-u.s.-2018/tables/table-4/table-4.xls/output.xls',
'https://ucr.fbi.gov/crime-in-the-u.s/2019/crime-in-the-u.s.-2019/tables/table-4/table-4.xls/output.xls']

Crime_datasets = ['2009_2010.xls', '2011_2012.xls', '2013_2014.xls', '2015_2016.xls', '2017_2018.xls', '2019.xls']
os.path.exists
os.path.exists(Crime_datasets)

 #for i in urls:
  # if(!file.exists(Crime.datasets[i])){
#   download.file(urls[i], Crime.datasets[i], mode = 'wb') # download xls from FBI website



NameError: name 'path_to_file' is not defined